In [480]:
reset()

In [481]:
load('../common_lib.sage')

In [482]:
preparser(False)

In [199]:
%%writefile ../MatRep.sage
def MatRep(param=(3,2), seed=None):
    """
    A quiz generator for finding the matrix representation
    
    INPUT:
    
    - ``param`` -- tuple `(dim, bound)`; 
      dim is the order of the A matrix  
      bound is the bound for random_int_matrix  
      and random_good_matrix
      
    - ``seed`` -- integer;  you may give a seed for  
      the random process if  you like.  Otherwise  
      the default is ``None`` and a random number  
      will be chosen.
      
    OUTPUT:
    
    three strings: QUESTION, SOLUTION, CHECKCODE
    """
    dim, bound = param
    random.seed(seed)

    A = random_int_matrix(dim, dim, bound=bound)
    B = random_good_matrix(dim, dim, dim, bound=bound)

    QUESTION = r"""\newcommand{\bv}{{\bf v}}
\newcommand{\rep}{\operatorname{Rep}} 
   
Let 
\[A = %(A)s \text{ and }
B = %(B)s.\]
Suppose $f:\mathbb{R}^%(dim)s\rightarrow\mathbb{R}^%(dim)s$ is a homomorphism  
defined by $f(\bv) = A\bv$ for all $\bv\in\mathbb{R}^%(dim)s$  
and $\mathcal{B}$ is a basis of $\mathbb{R}^%(dim)s$ composed of the columns of $B$.  
Find $\rep_{\mathcal{B},\mathcal{B}}(f)$.

\bigskip
Check code $=$ (sum of all entries of $\rep_{\mathcal{B},\mathcal{B}}(f)$) mod $10$
    """%{'A': latex_matrix(A), 
         'B': latex_matrix(B), 
         'dim': dim
        }
    
    AB = A * B
    Av_stg = ', '.join([r'A\bv_%s = %s'%(j+1, latex_matrix(AB[:,j])) for j in range(dim-1)]) + \
             r', \text{ and } A\bv_%s = %s'%(dim, latex_matrix(AB[:,dim-1]))
    BinvAB = B.inverse() * AB
    repAv_stg = ', '.join([r'\rep_{\mathcal{B}}(A\bv_%s) = %s'%(j+1, latex_matrix(BinvAB[:,j])) for j in range(dim-1)]) + \
                r', \text{ and } \rep_{\mathcal{B}}(A\bv_%s) = %s'%(dim, latex_matrix(AB[:,dim-1]))
    
    checkcode = sum(BinvAB[i,j] for i in range(dim) for j in range(dim)) % 10
    CHECKCODE = "%s"%checkcode
    
    SOLUTION = r"""Let $\bv_j$ be the $j$-th column of $B$.  
Then $f(\bv_j)$'s can be computed as  
\[%(Av_stg)s\]
and $\rep_{\mathcal{B}}(f(\bv_j))$'s are
\[%(repAv_stg)s.\]  
Thus, 
\[\rep_{\mathcal{B},\mathcal{B}}(f) = \rboxed{%(BinvAB)s}.\]  

\bigskip 
Check code $=$ (sum of all entries of $\rep_{\mathcal{B},\mathcal{B}}(f)$) mod $10$ $= %(checkcode)s$.
    """%({'Av_stg': Av_stg, 
          'repAv_stg': repAv_stg,
          'BinvAB': latex_matrix(BinvAB),
          'checkcode': checkcode
         })
    
    return QUESTION, SOLUTION, CHECKCODE

Overwriting ../MatRep.sage


In [225]:
%%writefile ../FindDet.sage
def FindDet(param=(4,3), seed=None):
    """
    A quiz generator for finding the determinant
    
    INPUT:
    
    - ``param`` -- tuple `(dim, bound)`; 
      dim is the order of the A matrix  
      bound is the bound for random_int_matrix  
      
    - ``seed`` -- integer;  you may give a seed for  
      the random process if  you like.  Otherwise  
      the default is ``None`` and a random number  
      will be chosen.
      
    OUTPUT:
    
    three strings: QUESTION, SOLUTION, CHECKCODE
    """
    dim, bound = param
    random.seed(seed)

    A = random_int_matrix(dim, dim, bound=bound)

    QUESTION = r"""\newcommand{\bv}{{\bf v}}
\newcommand{\rep}{\operatorname{Rep}} 
   
Let 
\[A = %(A)s.\]
Find $\det(A)$.

\bigskip
Check code $=$ $\det(A)$ mod $10$
    """%{'A': latex_matrix(A)
        }
    
    Adet = A.determinant()
    
    checkcode = Adet % 10
    CHECKCODE = "%s"%checkcode
    
    SOLUTION = r"""You may use Laplace's expansion or the permutation expansion to compute the determinant.  
The determinant of $A$ is $\rboxed{%(Adet)s}$.

\bigskip 
Check code $=$ $\det(A)$ mod $10$ $= %(checkcode)s$.
    """%({'Adet': Adet, 
          'checkcode': checkcode
         })
    
    return QUESTION, SOLUTION, CHECKCODE

Overwriting ../FindDet.sage


In [503]:
%%writefile ../AbsSumEigs.sage
def AbsSumEigs(param=(3,5), seed=None):
    """
    A quiz generator for finding the sum of absolute values of eigenvalues
    
    INPUT:
    
    - ``param`` -- tuple `(dim, bound)`; 
      dim is the order of the A matrix  
      bound is the bound for random_int_eigs_matrix  
      
    - ``seed`` -- integer;  you may give a seed for  
      the random process if  you like.  Otherwise  
      the default is ``None`` and a random number  
      will be chosen.
      
    OUTPUT:
    
    three strings: QUESTION, SOLUTION, CHECKCODE
    """
    dim, bound = param
    random.seed(seed)

    A = random_int_eigs_matrix(dim, bound=bound)

    QUESTION = r"""\newcommand{\bv}{{\bf v}}
\newcommand{\rep}{\operatorname{Rep}} 
   
Let 
\[A = %(A)s.\]  
Suppose the eigenvalues of $A$ are $\lambda_1,\ldots,\lambda_%(dim)s$.  
Find the value of $S = \sum_{i=1}^%(dim)s \lambda_i$, where $|\cdot |$ is the absolute value.

\bigskip
Check code $=$ $S$ mod $10$
    """%{'A': latex_matrix(A), 
         'dim': dim
        }
    
    p = A.characteristic_polynomial()
    eigs = A.eigenvalues()
    eigs_stg = ', '.join(['%s'%eig for eig in eigs])
    AbsSum = sum(map(abs, eigs))
    
    checkcode = AbsSum % 10
    CHECKCODE = "%s"%checkcode
    
    SOLUTION = r"""The characteristic polynomial of $A$ is 
\[%(p)s\]
and the eigenvalues are 
\[\{%(eigs_stg)s\}.\]  
Therefore, $S = \rboxed{%(AbsSum)s}$.

\bigskip 
Check code $=$ $\det(A)$ mod $10$ $= %(checkcode)s$.
    """%({'p': latex(p), 
          'eigs_stg': eigs_stg, 
          'AbsSum': AbsSum, 
          'checkcode': checkcode
         })
    
    return QUESTION, SOLUTION, CHECKCODE

Overwriting ../AbsSumEigs.sage


In [500]:
q, s, c = AbsSumEigs()

In [501]:
print(q)
print('---')
print(s)
print('---')
print(c)

\newcommand{\bv}{{\bf v}}
\newcommand{\rep}{\operatorname{Rep}} 
   
Let 
\[A = \begin{bmatrix}
 -18 & 30 & 70 & -10 \\ 
 -20 & 37 & 85 & -5 \\ 
 5 & -10 & -23 & 0 \\ 
 5 & -10 & -25 & 2 \\ 
\end{bmatrix}.\]  
Suppose the eigenvalues of $A$ are $\lambda_1,\ldots,\lambda_4$.  
Find the value of $S = \sum_{i=1}^4 \lambda_i$, where $|\cdot |$ is the absolute value.

\bigskip
Check code $=$ $S$ mod $10$
    
---
The characteristic polynomial of $A$ is 
\[x^{4} + 2 x^{3} - 11 x^{2} - 12 x + 36\]
and the eigenvalues are 
\[\{2, 2, -3, -3\}.\]  
Therefore, $S = \rboxed{10}$.

\bigskip 
Check code $=$ $\det(A)$ mod $10$ $= 0$.
    
---
0


In [491]:
'%s'%p

'x^3 + 3*x^2 + 3*x + 1'

In [492]:
latex(p)

x^{3} + 3 x^{2} + 3 x + 1

In [487]:
A = random_int_eigs_matrix(3,1)

In [488]:
A

[-1  0  0]
[ 0 -1  0]
[ 0  0 -1]

In [490]:
p = A.characteristic_polynomial()

In [486]:
A.eigenvalues()

[1, 1, -1, -1, -1]

Temporarily give up  
to generate symmetric matrices  
with integer eigenvalues

In [418]:
while True:
    bound = 5
    all_nz_numbers = list(range(-bound, bound+1))
    all_nz_numbers.remove(0)
    a00 = random.choice(all_nz_numbers)
    a10 = random.choice(all_nz_numbers)

    V = matrix([
        [a00, -a10, a00],
        [a10, a00, a10],
        [1, 0, -a00**2 - a10**2]
    ])
    # d1 = gcd([V[i,1] for i in range(2)])
    # d2 = gcd([V[i,2] for i in range(2)])
    # V[:,1] /= d1
    # V[:,2] /= d2

    D = diagonal_matrix([random.choice([1,2]),
                         -random.choice([1,2]),
                         -1
                        ])         

    A = V * D * V.transpose()
    eigs = A.eigenvalues()
    if sum(map(abs, eigs)) <= 100:
        break

In [419]:
A

[-1 -3  4]
[-3 -1 -4]
[ 4 -4 -2]

In [420]:
A.characteristic_polynomial()

x^3 + 4*x^2 - 36*x - 144

In [421]:
A.eigenvalues()

[6, -4, -6]

In [417]:
V

[-2  1 -2]
[-1 -2 -1]
[ 1  0 -5]

In [351]:
gcd([-2,2,0])

2

In [300]:
A

[ -8  -8 -18]
[ -8  -8  18]
[-18  18 -63]

In [301]:
V

[-2 -2 -2]
[ 2 -2  2]
[ 1  0 -8]